In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
# 코드에서 nms제외해야함

In [2]:
# !python3 -m pip install --pre-upgrade tensorrt
# !pip install pycuda

In [3]:
# !python3 -m pip install --pre --upgrade tensorrt
# !apt-get install tensorrt -y
# !wget https://developer.nvidia.com/downloads/compute/machine-learning/tensorrt/10.0.0/tars/TensorRT-10.0.0.6.Linux.x86_64-gnu.cuda-11.8.tar.gz

# python3 -m pip install tensorrt-10.0.0b6-cp38-none-linux_x86_64.whl
# export PATH=/path/to/tensorrt/bin:$PATH
# export LD_LIBRARY_PATH=<TensorRT-${version}/lib>:$LD_LIBRARY_PATH
# source ~/.bashrc
# https://docs.nvidia.com/deeplearning/tensorrt/install-guide/index.html#installing-tar
# https://da2so.tistory.com/61

In [4]:
# base_name = "./deploy_pillar_sfaw_3d/dsvt_3d_bb.onnx"
# --memPoolSize=workspace:4096
# --device=1
# --buildOnly
# --tacticSources=+CUDNN,+CUBLAS,-CUBLAS_LT,+EDGE_MASK_CONVOLUTIONS \
# --minShapes=src:3000x192,set_voxel_inds_tensor_shift_0:2x170x36,set_voxel_inds_tensor_shift_1:2x100x36,set_voxel_masks_tensor_shift_0:2x170x36,set_voxel_masks_tensor_shift_1:2x100x36,pos_embed_tensor:4x2x3000x192 \
# --optShapes=src:20000x192,set_voxel_inds_tensor_shift_0:2x1000x36,set_voxel_inds_tensor_shift_1:2x700x36,set_voxel_masks_tensor_shift_0:2x1000x36,set_voxel_masks_tensor_shift_1:2x700x36,pos_embed_tensor:4x2x20000x192 \
# --maxShapes=src:35000x192,set_voxel_inds_tensor_shift_0:2x1500x36,set_voxel_inds_tensor_shift_1:2x1200x36,set_voxel_masks_tensor_shift_0:2x1500x36,set_voxel_masks_tensor_shift_1:2x1200x36,pos_embed_tensor:4x2x35000x192 


# !trtexec --onnx=./deploy_pillar_sfaw_3d_origin/deploy_pillar_sfaw_3d_origin.onnx  --saveEngine=./deploy_pillar_sfaw_3d/dsvt_3d_bb_origin.engine --verbose   --fp16

# --optShapes=src:962x192,set_voxel_inds_tensor_shift_0:2x40x36,set_voxel_inds_tensor_shift_1:2x34x36,set_voxel_masks_tensor_shift_0:2x40x36,set_voxel_masks_tensor_shift_1:2x34x36,pos_embed_tensor:4x2x962x192 \
# --optShapes=src:20000x192,set_voxel_inds_tensor_shift_0:2x1000x36,set_voxel_inds_tensor_shift_1:2x700x36,set_voxel_masks_tensor_shift_0:2x1000x36,set_voxel_masks_tensor_shift_1:2x700x36,pos_embed_tensor:4x2x20000x192 \
# --maxShapes=src:35000x192,set_voxel_inds_tensor_shift_0:2x1500x36,set_voxel_inds_tensor_shift_1:2x1200x36,set_voxel_masks_tensor_shift_0:2x1500x36,set_voxel_masks_tensor_shift_1:2x1200x36,pos_embed_tensor:4x2x35000x192 


In [7]:
# torch.float32
# torch.Size([962, 192])
# torch.int32
# torch.Size([2, 40, 36])
# torch.int32
# torch.Size([2, 34, 36])
# torch.bool
# torch.Size([2, 40, 36])
# torch.bool
# torch.Size([2, 34, 36])
# torch.float32
# torch.Size([4, 2, 962, 192])

In [8]:
import numpy as np
import tensorrt as trt
import torch
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.models import build_network
from pcdet.datasets import build_dataloader
from pcdet.utils import common_utils
# import pycuda.driver as cuda
# import pycuda.autoinit
# import torch_tensorrt

# import pycuda.driver as cuda
# import pycuda.autoinit

fatal: detected dubious ownership in repository at '/mnt/nas2/users/eslim/workspace/OpenPCDet'
To add an exception for this directory, call:

	git config --global --add safe.directory /mnt/nas2/users/eslim/workspace/OpenPCDet


In [9]:
input_names = [
    'src',
    'set_voxel_inds_tensor_shift_0', 
    'set_voxel_inds_tensor_shift_1', 
    'set_voxel_masks_tensor_shift_0', 
    'set_voxel_masks_tensor_shift_1',
    'pos_embed_tensor'
]
output_names = ["output",]
trt_path = './deploy_pillar_sfaw_3d_origin/dsvt_3d_bb_origin_2.engine'


In [10]:
####### load model #######
# cfg_file = "./cfgs/dsvt_models/dsvt_plain_1f_onestage.yaml"
# cfg_from_yaml_file(cfg_file, cfg)
# if os.path.exists('./deploy_files')==False:
#     os.mkdir('./deploy_files')
# log_file = './deploy_files/log_trt.log'

cfg_file = "./pillar/config.yaml"
cfg_from_yaml_file(cfg_file, cfg)
if os.path.exists('./deploy_pillar_sfaw_3d_origin')==False:
    os.mkdir('./deploy_pillar_sfaw_3d_origin')
log_file = './deploy_pillar_sfaw_3d_origin/log_trt.log'

logger = common_utils.create_logger(log_file, rank=0)
test_set, test_loader, sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=1,
    dist=False, workers=8, logger=logger, training=False
)

model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=test_set)
ckpt = "./pillar/model.pth"

model.load_params_from_file(filename=ckpt, logger=logger, to_cpu=False, pre_trained_path=None)
model.eval()
model.cuda()
####### load model #######

####### read input #######
batch_dict = torch.load("/mnt/nas2/users/eslim/onnx/sfaw/input_dict.pth", map_location="cuda")
inputs = batch_dict
inputs = model.vfe(inputs)


2024-04-16 00:39:30,905   INFO  Loading Custom dataset.
2024-04-16 00:39:30,907   INFO  Total samples for CUSTOM dataset: 0
2024-04-16 00:39:33,048   INFO  ==> Loading parameters from checkpoint ./pillar/model.pth to GPU
2024-04-16 00:39:33,176   INFO  ==> Checkpoint trained from version: pcdet+0.6.0+255db8f
2024-04-16 00:39:33,210   INFO  ==> Done (loaded 391/391)


In [11]:
# #TODO : input data , preprocessing

# cfg_file = "./pillar/config.yaml"
# cfg_from_yaml_file(cfg_file, cfg)
# if os.path.exists('./deploy_pillar_sfaw_3d_origin')==False:
#     os.mkdir('./deploy_pillar_sfaw_3d_origin')
# log_file = './deploy_pillar_sfaw_3d_origin/log_trt.log'

# logger = common_utils.create_logger(log_file, rank=0)
# test_set, test_loader, sampler = build_dataloader(
#     dataset_cfg=cfg.DATA_CONFIG,
#     class_names=cfg.CLASS_NAMES,
#     batch_size=1,
#     dist=False, workers=8, logger=logger, training=False
# )

# model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=test_set)


# batch_dict = torch.load("/mnt/nas2/users/eslim/onnx/sfaw/input_dict.pth", map_location="cuda")
# inputs = batch_dict
# inputs = model.vfe(inputs)
# DSVT_Backbone = model.backbone_3d
# dsvtblocks_list = DSVT_Backbone.stage_0
# layer_norms_list = DSVT_Backbone.residual_norm_stage_0
# inputs = model.vfe(inputs)
# voxel_info = DSVT_Backbone.input_layer(inputs)
# voxel_feat = voxel_info['voxel_feats_stage0']

# pillar_features = inputs['voxel_features']
# set_voxel_inds_list = [[voxel_info[f'set_voxel_inds_stage{s}_shift{i}'] for i in range(2)] for s in range(1)]
# set_voxel_masks_list = [[voxel_info[f'set_voxel_mask_stage{s}_shift{i}'] for i in range(2)] for s in range(1)]
# pos_embed_list = [[[voxel_info[f'pos_embed_stage{s}_block{b}_shift{i}'] for i in range(2)] for b in range(4)] for s in range(1)]

# output = voxel_feat
# inputs = dict(
#         src=output,
#         set_voxel_inds_tensor_shift_0=set_voxel_inds_list[0][0].int(),
#         set_voxel_inds_tensor_shift_1=set_voxel_inds_list[0][1].int(),
#         set_voxel_masks_tensor_shift_0=set_voxel_masks_list[0][0],
#         set_voxel_masks_tensor_shift_1=set_voxel_masks_list[0][1],
#         pos_embed_tensor=torch.stack([torch.stack(v, dim=0) for v in pos_embed_list[0]], dim=0),
#     )

In [12]:
output.size()
output
# set_voxel_inds_list[0][0].int().size()

tensor([[0.0000, 0.0458, 0.0000,  ..., 0.0303, 0.0000, 0.0000],
        [0.0000, 0.0686, 0.0000,  ..., 0.0079, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.5312, 0.4485, 0.0000,  ..., 0.0691, 0.0000, 0.5523],
        [0.0000, 0.4938, 0.0000,  ..., 0.0557, 0.0000, 0.0000],
        [0.0000, 0.4529, 0.0000,  ..., 0.0210, 0.0000, 0.0000]],
       device='cuda:0', grad_fn=<CloneBackward0>)

In [13]:
# tensor([[-1.0312, -0.2234, -0.2581,  ..., -1.2207,  0.0303, -0.2534],
#         [ 0.0000,  0.0686,  0.0000,  ...,  0.0079,  0.0000,  0.0000],
#         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
#         ...,
#         [ 0.5312,  0.4485,  0.0000,  ...,  0.0691,  0.0000,  0.5523],
#         [ 0.0000,  0.4938,  0.0000,  ...,  0.0557,  0.0000,  0.0000],
#         [ 0.0000,  0.4529,  0.0000,  ...,  0.0210,  0.0000,  0.0000]],
#        device='cuda:0', grad_fn=<CloneBackward0>)

In [14]:
def load_trt_engine(path: str) -> trt.ICudaEngine:
    """Deserialize TensorRT engine from disk.

    Args:
        path (str): The disk path to read the engine.

    Returns:
        tensorrt.ICudaEngine: The TensorRT engine loaded from disk.
    """
    # load_tensorrt_plugin()
    with trt.Logger() as logger, trt.Runtime(logger) as runtime:
        with open(path, mode='rb') as f:
            engine_bytes = f.read()
        engine = runtime.deserialize_cuda_engine(engine_bytes)
        print(f"TensorRT engine {path} successfully loaded.")
        return engine

In [15]:
trt.init_libnvinfer_plugins(None, '')
if isinstance(trt_path, str):
    engine = load_trt_engine(trt_path)

if not isinstance(engine, trt.ICudaEngine):
    raise TypeError(f'`engine` should be str or trt.ICudaEngine, \
        but given: {type(engine)}')

context = engine.create_execution_context()

TensorRT engine ./deploy_pillar_sfaw_3d_origin/dsvt_3d_bb_origin_2.engine successfully loaded.
[04/16/2024-00:39:35] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading


In [16]:
# context.set_tensor_address("src", ptr)


In [93]:
# inputs.items()
engine.get_tensor_name(0)
# engine.get_tensor_name(1)
# engine.get_tensor_name(2)
# engine.get_tensor_name(3)
# engine.get_tensor_name(4)
# engine.get_tensor_name(5)
engine.get_tensor_name(6)



'output'

In [18]:
src_shape = engine.get_tensor_profile_shape("src", 0)[0]
engine.get_tensor_profile_shape("output", 0)

[(962, 192), (962, 192), (962, 192)]

In [19]:
# engine.get_tensor_profile_values(0,"src")
# input_host_mem = cuda.pagelocked_empty(trt.volume(src_shape) * trt.float32.itemsize)

bindings = [None] * (len(input_names) + len(output_names))



In [84]:
# context.execute_async_v3(
#             bindings,
#             torch.cuda.current_stream().cuda_stream,
#         )


In [80]:
engine.get_profile_shape(0,"src")
engine.get_tensor_location("src")
engine.get_tensor_format("src")
engine.get_tensor_name(1)
# engine.get_tensor_profile_values(0,"src")

/tmp/ipykernel_2586124/3761975307.py:1: DeprecationWarning: Use get_tensor_profile_shape instead.
  engine.get_profile_shape(0,"src")


<TensorFormat.LINEAR: 0>

In [90]:
# profile = self.engine.get_profile_shape(profile_id, input_name)
# assert input_tensor.dim() == len(
#     profile[0]), 'Input dim is different from engine profile.'
# for s_min, s_input, s_max in zip(eval(repr(profile[0])), input_tensor.shape,
#                                  eval(repr(profile[2]))):
#     assert s_min <= s_input <= s_max, \
#         f'Input shape of {input_name} should be between ' \
#         + f'{profile[0]} and {profile[2]}' \
#         + f' but get {tuple(input_tensor.shape)}.'

In [89]:
bindings = [None] * (len(input_names) + len(output_names))

# https://github.dev/Haiyang-W/DSVT , dsvt.py -> 384line
profile_id = 0
idx = 0
for input_name, input_tensor in inputs.items():    
    profile = engine.get_profile_shape(profile_id,input_name)
    print(f'profile : {profile}')
    print(f'type : {input_tensor.dtype}')
    print(f'shape : {input_tensor.size()}')
    context.set_input_shape(input_name,tuple(input_tensor.shape))
    bindings[idx] = input_tensor.contiguous().data_ptr()
    idx+=1
# get_binding_index

profile : [(962, 192), (962, 192), (962, 192)]
type : torch.float32
shape : torch.Size([962, 192])
profile : [(2, 40, 36), (2, 40, 36), (2, 40, 36)]
type : torch.int32
shape : torch.Size([2, 40, 36])
profile : [(2, 34, 36), (2, 34, 36), (2, 34, 36)]
type : torch.int32
shape : torch.Size([2, 34, 36])
profile : [(2, 40, 36), (2, 40, 36), (2, 40, 36)]
type : torch.bool
shape : torch.Size([2, 40, 36])
profile : [(2, 34, 36), (2, 34, 36), (2, 34, 36)]
type : torch.bool
shape : torch.Size([2, 34, 36])
profile : [(4, 2, 962, 192), (4, 2, 962, 192), (4, 2, 962, 192)]
type : torch.float32
shape : torch.Size([4, 2, 962, 192])


/tmp/ipykernel_2586124/1391992233.py:7: DeprecationWarning: Use get_tensor_profile_shape instead.
  profile = engine.get_profile_shape(profile_id,input_name)


In [91]:
bindings

[139711251957248,
 139712085820416,
 139712085832192,
 139711243770880,
 139711243773952,
 139711206561280,
 None]

In [92]:
bindings[-1] = output.data_ptr()


# for output_name in output_names:
    # print(output_name)
    # idx = self.engine.get_binding_index(output_name)
    # dtype = torch_dtype_from_trt(self.engine.get_binding_dtype(idx))
    # shape = eval(repr(self.context.get_binding_shape(idx)))

    # device = torch_device_from_trt(self.engine.get_location(idx))
    # output = torch.empty(size=shape, dtype=dtype, device=device)
    # outputs[output_name] = output
    # bindings[idx] = output.data_ptr()


output


In [25]:
bindings

[139711251957248,
 139712085820416,
 139712085832192,
 139711243770880,
 139711243773952,
 139711206561280,
 139711251957248]

In [26]:
context.execute_v2(
            bindings
        )


True

In [27]:
output.size()


torch.Size([962, 192])

In [28]:
output

tensor([[-0.9653, -0.2267, -0.1353,  ..., -1.1621, -0.1450, -0.2219],
        [-1.0186, -0.1864, -0.1697,  ..., -1.1982, -0.1243, -0.2854],
        [-1.0752, -0.1381, -0.1663,  ..., -1.0742,  0.1079, -0.2844],
        ...,
        [-0.3411,  0.4285, -0.3716,  ..., -0.9014, -0.8604, -0.0605],
        [-0.6758,  0.6250, -0.0422,  ..., -1.0859, -1.0088, -0.3896],
        [-0.7290,  0.2339,  0.1506,  ..., -1.0791, -0.2252, -0.5078]],
       device='cuda:0', grad_fn=<CloneBackward0>)

In [39]:
# context.execute_async_v3(bindings[0])

In [ ]:
####


In [30]:
# # engine = load_engine(engine_file_path)
# # Load the TensorRT engine from the file
# with open(engine_file_path, 'rb') as f, trt.Runtime(trt.Logger(trt.Logger.WARNING)) as runtime:
#     engine = runtime.deserialize_cuda_engine(f.read())
#     context = engine.create_execution_context()
# # engine


In [31]:
# num_bindings = engine.num_bindings
# binding_info = []
# for i in range(num_bindings):
#     binding_info.append({
#         'index': i,
#         'name': engine.get_binding_name(i),
#         'shape': engine.get_binding_shape(i),
#         'dtype': trt.nptype(engine.get_binding_dtype(i))
#     })

In [ ]:
context = engine.create_execution_context()

# 입력 및 출력 버퍼를 할당합니다.
input_shape = engine.get_binding_shape(0)
output_shape = engine.get_binding_shape(1)
input_host_mem = cuda.pagelocked_empty(trt.volume(input_shape) * trt.float32.itemsize)
output_host_mem = cuda.pagelocked_empty(trt.volume(output_shape) * trt.float32.itemsize)
input_device_mem = cuda.mem_alloc(input_host_mem.nbytes)
output_device_mem = cuda.mem_alloc(output_host_mem.nbytes)

# 이미지 전처리
image = preprocess_image(image_path, input_shape)

# 입력을 GPU 메모리로 복사합니다.
cuda.memcpy_htod(input_device_mem, image.flatten())

# 추론 실행
bindings = [int(input_device_mem), int(output_device_mem)]
context.execute_v2(bindings)

# 결과를 GPU에서 CPU로 복사합니다.
cuda.memcpy_dtoh(output_host_mem, output_device_mem)

In [44]:
input_dict = {}
input_dict['data_dict']={}
input_dict['data_dict']['voxel_features'] = ort_outs[0]
input_dict['data_dict']['use_lead_xyz'] = ort_outs[1]
input_dict['data_dict']['batch_size'] = ort_outs[2]
input_dict['data_dict']['pillar_features'] = ort_outs[3]
input_dict['data_dict']['voxel_coords'] = ort_outs[4]
input_dict['data_dict']['spatial_features'] = ort_outs[5]


In [45]:
base_name = "./deploy_pillar_sfaw_2d/dsvt_2d_bb"
ts_path = f"{base_name}.ts"
onnx_path = f"{base_name}.onnx"

ort_session = ort.InferenceSession(onnx_path)

ort_inputs = {
    'voxel_features' : input_dict['data_dict']['voxel_features'],
    'use_lead_xyz' : input_dict['data_dict']['use_lead_xyz'],
    'batch_size' :  input_dict['data_dict']['batch_size'],
    'pillar_features' : input_dict['data_dict']['pillar_features'],
    'voxel_coords' : input_dict['data_dict']['voxel_coords'],
    'spatial_features' : input_dict['data_dict']['spatial_features']
}
ort_outs = ort_session.run(None, ort_inputs)
ort_outs


[array([[-1.09687102e+00, -8.87880400e-02,  1.56932503e-01, ...,
         -1.04772103e+00,  3.80000025e-02, -1.98598042e-01],
        [-1.09640002e+00, -7.08313286e-02,  1.60521924e-01, ...,
         -1.08235085e+00,  4.23637405e-02, -1.92351431e-01],
        [-1.08620584e+00, -1.00942045e-01,  1.78571254e-01, ...,
         -1.05504239e+00,  2.63126958e-02, -1.59484476e-01],
        ...,
        [-3.60771865e-01,  2.12830842e-01, -1.58340424e-01, ...,
         -1.23321116e+00, -1.63467184e-01,  1.67723268e-01],
        [-6.18592083e-01,  3.51619005e-01, -1.11299597e-01, ...,
         -1.37231803e+00, -1.09309152e-01, -5.45044169e-02],
        [-6.13398969e-01,  3.21275711e-01, -1.22284256e-02, ...,
         -1.24491286e+00,  6.93649054e-04, -1.37990996e-01]], dtype=float32),
 array([1.], dtype=float32),
 array(1, dtype=int64),
 array([[-1.09687102e+00, -8.87880400e-02,  1.56932503e-01, ...,
         -1.04772103e+00,  3.80000025e-02, -1.98598042e-01],
        [-1.09640002e+00, -7.083132

In [46]:
#### Head

In [48]:
# data_dict = {}
input_dict = {}
input_dict['data_dict']={}
input_dict['data_dict']['voxel_features'] = ort_outs[0]
input_dict['data_dict']['use_lead_xyz'] = ort_outs[1]
input_dict['data_dict']['batch_size'] = ort_outs[2]
input_dict['data_dict']['pillar_features'] = ort_outs[3]
input_dict['data_dict']['voxel_coords'] = ort_outs[4]
input_dict['data_dict']['spatial_features'] = ort_outs[5]
input_dict['data_dict']['spatial_features_2d'] = ort_outs[6]



In [50]:
# # test onnx
base_name = "./deploy_pillar_sfaw_head/dsvt_head"
ts_path = f"{base_name}.ts"
onnx_path = f"{base_name}.onnx"


ort_session = ort.InferenceSession(onnx_path)

ort_inputs = {
    'voxel_features' : input_dict['data_dict']['voxel_features'],
    'use_lead_xyz' : input_dict['data_dict']['use_lead_xyz'],
    'batch_size' :  input_dict['data_dict']['batch_size'],
    'pillar_features' : input_dict['data_dict']['pillar_features'],
    'voxel_coords' : input_dict['data_dict']['voxel_coords'],
    'spatial_features' : input_dict['data_dict']['spatial_features'],
    'spatial_features_2d' : input_dict['data_dict']['spatial_features_2d']
}

ort_outs = ort_session.run(None, ort_inputs)
ort_outs

ValueError: Required inputs (['onnx::Identity_0', 'onnx::Identity_1', 'onnx::Identity_2', 'onnx::Identity_3', 'onnx::Identity_4', 'onnx::Identity_5', 'input.1']) are missing from input feed (['voxel_features', 'use_lead_xyz', 'batch_size', 'pillar_features', 'voxel_coords', 'spatial_features', 'spatial_features_2d']).